# CW2.1:  Interpreter for FUNC

Your overall task is to develop a full interpreter and compiler for the programming language given below, called ``FUNC``.
This overall task is composed of five parts: Writing an interpreter, basic code generation, advanced code generation, a lexer, and a parser.


**CW 2 Part I** consists of two parts: 
- Writing an interpreter for the parts not requiring functions (6 points)
- Writing an interpreter for the parts requiring functions  (4 points)

If you have any questions, use the labs slots or ask Kathrin & the Lab Helpers.

**IMPORTANT** 
Compiler errors: All code you submit must compile. Programs that do not compile will receive an automatic zero.
- If you are having trouble getting your assignment to compile, please visit consulting hours.
- If you run out of time, it is better to comment out the parts that do not compile, than hand in a more complete file that does not compile.

## Testing 

At the end of this file you'll find example program you can test your programs with. 
**You will want to write additional tests for intermediate steps.**

You can easily write tests to ensure that your program behaves as expected as follows:

In [1]:
assert ([2;3;5;5;2;1] (* Expected result *) 
= [2;3;5] @ [5;2;1] (* Calling your function *) ) ;; 

- : unit = ()


**The plagarism policy does not hold for this part of the coursework. 
Please feel free to share your tests with other students in the course.**

## Submission

Please submit a .zip file containing this notebook on Canvas until **Thu, 7th March**. 
Please ensure that you do not change the name or signature of the functions ``eval_exp``, ``eval_program``, etc. 

**Late Submissions.** See Canvas for F29LP's late-submission policy. 

**Plagarism.** All code (except tests) is subject to the course's plagarism policy. 

Happy coding!

## The Source Language: FUNC

The ``FUNC`` language has the following syntax: 

```
<program> ::= <methods> 
<methods> ::= <method>;[<methods>] 
<method> ::= method <id>([<args>]) [vars <args>] 
	begin <statements> [return <id>;] endmethod
<args> ::= <id>[,<args>] 
<statements> ::= <statement>;[<statements>] 
<statement> ::= <assign> | <if> | <while> | <rw>
<rw> ::= read <id> | write <exp>
<assign> ::= <id> := <exp>
<if> ::= if  <cond> then <statements> [else <statements>] endif 
<while> ::= while <cond> begin <statements> endwhile
<cond> ::= <bop> ( [<exps>] ) 
<bop> ::= less | lessEq | eq | nEq 
<exps> ::= <exp> [,<exps>] 
<exp> ::= <id>[( [<exps>] )] | <int> 
<int> is a natural number (no leading zeroes) 
<id> is any string starting with a character followed by characters or numbers (that is not already a keyword)
```

- Each program must have a function called ``main`` with no arguments and no return value. 
- All other functions may have an optional return value. If a function does not have a return value, they implicitly return `0`.
- You should support the following built-in functions - assume they have been defined; they accept two integers and return an integer:
     - ``plus``, which adds its arguments;
     - ``times``, which multiplies its arguments;
     - ``minus``, which subtracts its arguments;
     - ``divide``, which divides its arguments.
- All the boolean operators (``less``, ``lessEq``, ``eq``, ``nEq``) are also binary, i.e. take two arguments.
- The ``read`` command assumes that the given variable is an ``int`` variable.

##### Example 

The following example illustrates a valid FUNC program (more examples later in the document)

```
method pow(x, y) vars i, res
begin
    res := x; 
    i := 1; 
    while less(i,y)
    begin
        res := times(res,x);
        i := plus(i,1); 
    endwhile;
    write res;
    return res;
endmethod;

method main() vars a, b, x
begin
    a := 5; b := 2; 
    x := pow(b,a);
    if  eq(x,32) then write 1; else write 0; endif; 
endmethod;
```

Below you can see the abstract syntax of FUNC and a representation of the above program: 

In [2]:
type exp = Numb of int | Id of string | App of string * exp list

type bop = Less | LessEq | Eq | NEq 
type cond = C of bop * exp * exp

type statement =
  Assign of string * exp
| Read of string
| Write of exp 
| If of cond * statement list
| Ite of cond * statement list * statement list
| While of cond * statement list

type mmethod = M of string (* name of function *)
                * string list (* arguments *)
                * string list (* declarations *) 
                * statement list (* function body *)
                * string option (* possible return value *)

type program = P of mmethod list

let p1 : program  =
  P
    [M ("pow", ["x"; "y"], ["i"; "res"],
      [Assign ("res", Id "x"); Assign ("i", Numb 1);
       While (C (Less, Id "i", Id "y"),
        [Assign ("res", App ("times", [Id "res"; Id "x"]));
         Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
       Write (Id "res")],
      Some "res");
     M ("main", [], ["a"; "b"; "x"],
      [Assign ("a", Numb 5); Assign ("b", Numb 2);
       Assign ("x", App ("pow", [Id "b"; Id "a"]));
       Ite (C (Eq, Id "x", Numb 32), [Write (Numb 1)], [Write (Numb 0)])],
      None)]

type exp = Numb of int | Id of string | App of string * exp list


type bop = Less | LessEq | Eq | NEq


type cond = C of bop * exp * exp


type statement =
    Assign of string * exp
  | Read of string
  | Write of exp
  | If of cond * statement list
  | Ite of cond * statement list * statement list
  | While of cond * statement list


type mmethod =
    M of string * string list * string list * statement list * string option


type program = P of mmethod list


val p1 : program =
  P
   [M ("pow", ["x"; "y"], ["i"; "res"],
     [Assign ("res", Id "x"); Assign ("i", Numb 1);
      While (C (Less, Id "i", Id "y"),
       [Assign ("res", App ("times", [Id "res"; Id "x"]));
        Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
      Write (Id "res")],
     Some "res");
    M ("main", [], ["a"; "b"; "x"],
     [Assign ("a", Numb 5); Assign ("b", Numb 2);
      Assign ("x", App ("pow", [Id "b"; Id "a"]));
      Ite (C (Eq, Id "x", Numb 32), [Write (Numb 1)], [Write (Numb 0)])],
     None)]


As in the lectures, we will use environments to look up variables. 
As there are no block statements, a simple dictionary suffices.

In [3]:
exception RuntimeError of string

(* This will define maps with strings as key *)
module Env = Map.Make(String)

(* Env.empty denotes the empty environment. 
We can add elements to an environment via Env.add.
This is the environment which only binds “a” to 3. *)
let example_env = Env.add "a" 3 Env.empty;;

(* We can look up elements in an environment via Env.find.
Env.find throws an exception if the key does not exist.*)
Env.find "a" example_env;;

let update_env (x : string) v env = Env.add x v env 
let lookup_env (x : string) env = Env.find x env 

exception RuntimeError of string


module Env :
  sig
    type key = String.t
    type 'a t = 'a Map.Make(String).t
    val empty : 'a t
    val is_empty : 'a t -> bool
    val mem : key -> 'a t -> bool
    val add : key -> 'a -> 'a t -> 'a t
    val update : key -> ('a option -> 'a option) -> 'a t -> 'a t
    val singleton : key -> 'a -> 'a t
    val remove : key -> 'a t -> 'a t
    val merge :
      (key -> 'a option -> 'b option -> 'c option) -> 'a t -> 'b t -> 'c t
    val union : (key -> 'a -> 'a -> 'a option) -> 'a t -> 'a t -> 'a t
    val compare : ('a -> 'a -> int) -> 'a t -> 'a t -> int
    val equal : ('a -> 'a -> bool) -> 'a t -> 'a t -> bool
    val iter : (key -> 'a -> unit) -> 'a t -> unit
    val fold : (key -> 'a -> 'b -> 'b) -> 'a t -> 'b -> 'b
    val for_all : (key -> 'a -> bool) -> 'a t -> bool
    val exists : (key -> 'a -> bool) -> 'a t -> bool
    val filter : (key -> 'a -> bool) -> 'a t -> 'a t
    val filter_map : (key -> 'a -> 'b option) -> 'a t -> 'b t
    val partition : (key -> 'a -> bool) 

val example_env : int Env.t = <abstr>


- : int = 3


val update_env : string -> 'a -> 'a Env.t -> 'a Env.t = <fun>


val lookup_env : string -> 'a Env.t -> 'a = <fun>


## Part 1: Interpreter for Basic Parts (6 points)

In the first part of the assignment, you will write an interpreter for the part that does not require arbitrary functions, i.e. only works for the following subpart of the syntax: 


```
<statements> ::= <statement>;[<statements>] 
<statement> ::= <assign> | <if> | <while> | <rw>
<rw> ::= read <id> | write <exp>
<assign> ::= <id> := <exp>
<if> ::= if  <cond> then <statements> [else <statements>] endif 
<while> ::= while <cond> begin <statements> endwhile
<cond> ::= <bop> ( [<exps>] ) 
<bop> ::= less | lessEq | eq | nEq 
<exps> ::= <exp> [,<exps>] 
<exp> ::= <id> | plus (<exp>, <exp>) | minus (<exp>, <exp>) | times (<exp>, <exp>) | divide (<exp>, <exp>) | <int> 
<int> is a natural number (no leading zeroes) 
<id> is any string starting with a character followed by characters or numbers (that is not already a keyword)
```

Later on, we will require values in the environment to be either an ``int`` or closures.
For now, values are simply integers:

In [12]:
type value = VInt of int

type value = VInt of int


# Write a mutually recursive evaluator for statements in FUNC. 

Your evaluator should contain, mutually recursively: 
-  a function ``eval_exp : exp -> value Env.t -> value`` evaluating expressions to values. 
Your expressions should correctly evaluate identifiers ``Id x`` and constants ``Numb n``, but for function arguments have to only evaluate ``App ("plus", [e1; e2])``, ``App ("minus", [e1; e2])``, ``App ("times", [e1; e2])`` and ``App ("divide", [e1; e2])``.
You can match directly on these cases, i.e. 

```
match e with 
| App ("plus", [e1; e2]) -> (* What it should evaluate to *)
```


Note that in these cases, no methods have to be executed - ``App ("plus", [VInt m; VInt n])`` simply evaluates to ``VInt (m + n)``.

All other function calls should throw an error message that they are not implemented: 
```
match e with 
... 
| App (f, es) -> raise (RuntimeError "Arbitrary functions not implemented.")
```

(You want to test that all tests until ``test_add`` are working before you continue.)
- a function ``eval_condexp: cond -> value Env.t -> int`` that evaluates conditional expressions. 
- a function ``eval_stmt: statement -> value Env.t -> value Env.t`` that evaluates a single statement.
As in SIMP, please ignore the ``Read`` primitive, i.e. add the case: 

```
    | Read _ -> raise (RuntimeError "Read not implemented.")
```

- a function ``eval_stmts: statement list -> value Env.t -> value Env.t`` that evaluates a list of statements. 

In [15]:
let rec eval_exp (e : exp) (env : value Env.t) : value = match e with 
    | Numb n ->  VInt n (* Transforms directly to type value. *)
    | Id x -> Env.find x env (* Uses predefined helper function to return the value associated with ID *)
    | App ("plus", [e1; e2]) -> (match eval_exp e1 env, eval_exp e2 env with 
                                | VInt m, VInt n -> VInt (m + n))
    | App ("times", [e1; e2]) -> (match eval_exp e1 env, eval_exp e2 env with 
                                | VInt m, VInt n -> VInt (m * n))
    | App ("minus", [e1; e2]) -> (match eval_exp e1 env, eval_exp e2 env with 
                                | VInt m, VInt n -> VInt (m - n))
    | App ("divide", [e1; e2]) -> (match eval_exp e1 env, eval_exp e2 env with 
                                | VInt m, VInt n -> 
                                    if n != 0 (* So OCaml doesn't divide by 0. *)
                                    then VInt (m / n) 
                                    else raise (RuntimeError "Divison by 0"))
    | App (f, es) -> raise (RuntimeError "Arbitrary functions not implemented.")

and eval_condexp (e : cond) (env : value Env.t) : int = match e with                         
    | C (op, e1, e2) -> let VInt v1 = eval_exp e1 env in (* Declaring values to eliminate verbosity. *)
                        let VInt v2 = eval_exp e2 env in
                        match op with (* Matches with all possible values of type bop. *)
                        | Less -> if v1 < v2 then 1 else 0
                        | LessEq -> if v1 <= v2 then 1 else 0
                        | Eq -> if v1 = v2 then 1 else 0
                        | NEq -> if v1 != v2 then 1 else 0

and eval_stmt (s : statement) (env : value Env.t) : value Env.t = match s with 
  | Assign (x, e) -> Env.add x (eval_exp e env) env (* Uses predefined helper function to assign ID to values. *)
  | Write e -> let VInt v = eval_exp e env in
      print_endline ("OUTPUT: " ^ string_of_int v); env
  | If (c, s) -> (match (eval_condexp c env) with 
                        | 1 -> (match s with 
                                | stmt :: stmt' -> eval_stmt stmt env (* Evaluates head statement of list. *)
                                | [] -> env)
                        | 0 -> env
                        | _ -> raise (RuntimeError "Error in condition of If"))
  | Ite (c, s1, s2) -> (match (eval_condexp c env) with 
                        | 1 -> (match s1 with 
                                | stmt :: stmt' -> eval_stmt stmt env 
                                | [] -> env)
                        | 0 -> (match s2 with 
                                | stmt :: stmt' -> eval_stmt stmt env 
                                | [] -> env)
                        | _ -> raise (RuntimeError "Error in condition of Ite"))
  | While (c, s) -> let rec loop env = (* Only computes if eval_condexp returns 1 (mapped to true). *)
                      if (match (eval_condexp c env) with 
                          | 1 -> true 
                          | 0 -> false
                          | _ -> raise (RuntimeError "Error in condition of While"))
                      then loop (eval_stmts s env) 
                      else env
                      in loop env
  | _ -> raise (RuntimeError "Unimplemented statement.")

and eval_stmts (stmts : statement list) (env : value Env.t) : value Env.t = match stmts with 
    | [] -> env
    | s :: stms' -> eval_stmts stms' (eval_stmt s env) 

val eval_exp : exp -> value Env.t -> value = <fun>
val eval_condexp : cond -> value Env.t -> int = <fun>
val eval_stmt : statement -> value Env.t -> value Env.t = <fun>
val eval_stmts : statement list -> value Env.t -> value Env.t = <fun>


## Test Cases Part 1
Here is a (not necessarily complete) set of test cases: 

In [6]:
(* let p1 : program  =
  P
    [M ("pow", ["x"; "y"], ["i"; "res"],
      [Assign ("res", Id "x"); Assign ("i", Numb 1);
       While (C (Less, Id "i", Id "y"),
        [Assign ("res", App ("times", [Id "res"; Id "x"]));
         Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
       Write (Id "res")],
      Some "res");
     M ("main", [], ["a"; "b"; "x"],
      [Assign ("a", Numb 5); Assign ("b", Numb 2);
       Assign ("x", App ("pow", [Id "b"; Id "a"]));
       Ite (C (Eq, Id "x", Numb 32), [Write (Numb 1)], [Write (Numb 0)])],
      None)] *)

let env = update_env "x" (VInt 2) (update_env "y" (VInt 5) Env.empty)

let test_numb =  eval_exp (Numb 3) env
(* Should yield VInt 3. *)

let test_id =  eval_exp (Id "x") env
(* Should yield VInt 2. *)

let test_add = eval_exp (App ("plus", [Id "x"; Numb 3])) env
(* Should yield VInt 5. *)

let test_stmt_write = eval_stmt (Write (Numb 3)) env
(* This should have: "OUTPUT: 3" *)

let test_stmt_if = eval_stmt (If (C (Eq, Numb 2, Numb 2), [Write (Numb 3)])) env
(* This should have: "OUTPUT: 3" *)

let test_stmts = eval_stmts [Assign ("res", Id "x"); Assign ("i", Numb 1);
       While (C (Less, Id "i", Id "y"),
        [Assign ("res", App ("times", [Id "res"; Id "x"]));
         Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
       Write (Id "res")] env  
       
(* This should have: "OUTPUT: 32". *)

val env : value Env.t = <abstr>


val test_numb : value = VInt 3


val test_id : value = VInt 2


val test_add : value = VInt 5


OUTPUT: 3


val test_stmt_write : value Env.t = <abstr>


OUTPUT: 3


val test_stmt_if : value Env.t = <abstr>


OUTPUT: 32


val test_stmts : value Env.t = <abstr>


# Part 2: Interpreter Including Methods (4 points)

In the second part, you extend the interpreter with methods. 

Values can now be either integers or closures: 

In [7]:
type value = VInt of int | Closure of string list * statement list * string option 

type value =
    VInt of int
  | Closure of string list * statement list * string option


A **closure** describes a method, to be evaluated once we know what expressions the variables are bound to. It consists of the **argument names**, a list of statements called the **function body** and an optional return variable.

Let us look at the example of: 

```
method pow(x, y) vars i, res
begin
    res := x; 
    i := 1; 
    while less(i,y)
    begin
        res := times(res,x);
        i := plus(i,1); 
    endwhile;
    write res;
    return res;
endmethod;

method main() vars a, b, x
begin
    a := 5; b := 2; 
    x := pow(b,a);
    if  eq(x,32) then write 1; else write 0; endif; 
endmethod;
```

This is what happens during evaluation:
1. "pow" is bound to the closure of ``pow``, i.e. 
    ``Closure (["x"; "y"], "res := x ... write res", Some res)``. 
2. "main" is bound to the closure of ``main``, i.e. 
    ``Closure ([], "a := 5... endif", None)``.
3. The main function, ``App ("main", [])`` is evaluated in the enviroment ``env`` where pow/main are bound to their closure. As the main function has no arguments, no variables are bound to expressions. Evaluating ``App ("main", [])`` means that we 
    1. Look up the closure of "main" in the environment, ensuring it's actually a closure. (If it's not a closure, throw a Runtime Error.) 
    2. Potentially update the environment to bind the arguments of "main" to the handed over expressions. In this case, main does not take any arguments.
    3. Execute the statements of "main". Here, we first enhance the environment to env' and bind "a" to ``VInt 5`` and b to ``VInt 2``. Then, we call "pow(b, a)". 
        1. We look up the closure of "pow" in the environment, ensuring it's actually a closure. (If it's not a closure, throw a Runtime Error.)
        2. We update the environment to an environment env'' and bind "x" to the value of b, i.e. ``VInt 2``, and "y" to the value of "a", i.e. ``VInt 5``. This means the environment now contains "main", "pow", "a", "b", "x", and "y".
        3. We evaluate the statements of "pow" in this updated environment env'' to a new environment in which for example res is bound to 32. As res is the variable in the return statement, we evaluate to ``VInt 32`` in this most recent environment.
    4. We bind "x" to ``VInt 32``, and continue with the environment before calling ``pow(b, a)``, i.e. env'. 
    5. We write 1.

Your evaluator should do the following modifications to the code in part 1:

- You will want to start by copying your code from part 1.
- You might have to change parts where you are matching on values as functions might also be closures.
- You require a function ``eval_method : mmethod -> value Env.t -> value Env.t`` that adds the closure corresponding to the method to the environment. Note that *nothing* is evaluated at this point, but we just do a binding, i.e. ``eval_method (M ("pow", ["x"; "y"], ["i"; "res"], [Assign ("res", Id "x") ...], Some "res"))`` should yield the updated environment where "pow" is bound to ``Closure (["x"; "y"], [Assign ("res", Id "x") ...], Some res)``.

- You require a function ``eval_methods : mmethod list -> value Env.t -> value Env.t`` that repeats the above step to a list of methods. 
(Test that ``test_method`` and ``test_methods`` are working before you continue.)

- The function ``eval_program : program -> value`` should first evaluates the list of methods, and then evaluates `` eval_exp (App ("main", [])) env`` in this updated environment.

- You have to modify ``eval_exp`` for applications: 

```
 | App (f, es) -> ...
```
    
You might want to first start by getting this to work if ``es`` is empty. 
In this case you simply evaluate the body of *f*. 

(Test that ``test_no_arguments`` is working.)

- In the case of arguments, you have to first evaluate all expressions ``es`` (you can get a list of values using the ``List.map`` primitive, ``List.map (fun x -> eval_exp x env) es`` which evaluates all elements in ``es`` in the environment ``env``). You then have to bind these values to the arguments of "f", which you can find in its closure, to a new environment env'. You might require a new function for this part.
You then evaluate the body of f, i.e. use ``eval_stmts`` on ``es`` in the new environment env'.

In [17]:
(* Only App(f, es) is different in this part. *)
let rec eval_exp (e : exp) (env : value Env.t) : value = match e with 
    | Numb n ->  VInt n
    | Id x -> Env.find x env
    | App ("plus", [e1; e2]) -> (match eval_exp e1 env, eval_exp e2 env with 
                                | VInt m, VInt n -> VInt (m + n)
                                | _, _ -> raise (RuntimeError "Not integer values"))
    | App ("times", [e1; e2]) -> (match eval_exp e1 env, eval_exp e2 env with 
                                | VInt m, VInt n -> VInt (m * n)
                                | _, _ -> raise (RuntimeError "Not integer values"))
    | App ("minus", [e1; e2]) -> (match eval_exp e1 env, eval_exp e2 env with 
                                | VInt m, VInt n -> VInt (m - n)
                                | _, _ -> raise (RuntimeError "Not integer values"))
    | App ("divide", [e1; e2]) -> (match eval_exp e1 env, eval_exp e2 env with 
                                | VInt m, VInt n -> 
                                    if n != 0 
                                    then VInt (m / n) 
                                    else raise (RuntimeError "Divison by 0")
                                | _, _ -> raise (RuntimeError "Not integer values"))
    | App (f, es) -> let arg_vals = List.map (fun e -> eval_exp e env) es in 
                     let closure = lookup_env f env in
                     (match closure with
                     | Closure (f_args, f_body, return_opt) -> 
                         if List.length f_args <> List.length es (* Tests to stop program from crashing. *)
                         then raise (RuntimeError "Incorrect number of arguments")
                         else let local_env = List.fold_left2 (fun env' arg val' -> update_env arg val' env') env f_args arg_vals in
                              let result_env = eval_stmts f_body local_env in
                              (match return_opt with (* Allows for optional return value as specification requires. *)
                              | Some return_var -> (match Env.find_opt return_var result_env with
                                                   | Some v -> v
                                                   | None -> raise (RuntimeError "No return value"))
                              | None -> raise (RuntimeError "Function did not return a value"))
                     | _ -> raise (RuntimeError "Function application not supported"))

(* This function is the exact same in part 1. *)
and eval_condexp (e : cond) (env : value Env.t) : int = match e with                          
    | C (op, e1, e2) -> 
        (match eval_exp e1 env, eval_exp e2 env with
         | VInt v1, VInt v2 ->
            (match op with
            | Less -> if v1 < v2 then 1 else 0
            | LessEq -> if v1 <= v2 then 1 else 0
            | Eq -> if v1 = v2 then 1 else 0
            | NEq -> if v1 != v2 then 1 else 0)
         | _, _ -> raise (RuntimeError "Expected integer values in condition expression"))

(* Most of this function is the same to part 1. *)
and eval_stmt (s : statement) (env : value Env.t) : value Env.t = match s with 
  | Assign (x, e) -> Env.add x (eval_exp e env) env
  | Write e -> (* Had to change this case to be exhaustive due to addition of closures. *)
      (match eval_exp e env with
       | VInt v -> print_endline ("OUTPUT: " ^ string_of_int v); env
       | _ -> raise (RuntimeError "Expected integer value for Write"))
  | If (c, s) -> (match (eval_condexp c env) with 
                        | 1 -> (match s with 
                                | stmt :: stmt' -> eval_stmt stmt env 
                                | [] -> env)
                        | 0 -> env
                        | _ -> raise (RuntimeError "Error in condition of If")
                        )
  | Ite (c, s1, s2) -> (match (eval_condexp c env) with 
                        | 1 -> (match s1 with 
                                | stmt :: stmt' -> eval_stmt stmt env 
                                | [] -> env)
                        | 0 -> (match s2 with 
                                | stmt :: stmt' -> eval_stmt stmt env 
                                | [] -> env)
                        | _ -> raise (RuntimeError "Error in condition of Ite"))
  | While (c, s) -> let rec loop env =
                      if (match eval_condexp c env with
                          | 1 -> true
                          | 0 -> false
                          | _ -> raise (RuntimeError "Error in condition of While"))
                      then loop (eval_stmts s env) 
                      else env
                      in loop env
  | _ -> raise (RuntimeError "Unimplemented statement.")
  
(* This function is the exact same in part 1. *)
and eval_stmts (stmts : statement list) (env : value Env.t) : value Env.t = match stmts with 
    | [] -> env
    | s :: stms' -> eval_stmts stms' (eval_stmt s env) 
        
let eval_method (m : mmethod) (env : value Env.t) : value Env.t = match m with 
    | M (f, args, dcls, body, return) -> Env.add f (Closure (args, body, return)) env

(* Uses same general syntax as eval_stmts. *)
let rec eval_methods (ms : mmethod list) (env : value Env.t) : value Env.t =
    match ms with
    | [] -> env
    | m :: ms' -> 
        let env' = eval_method m env in
        eval_methods ms' env'

let eval_program p =
    let P ms = p in (* Program is of type method list. This looks at part of it. *)
    let global_env = eval_methods ms Env.empty in
    match List.find_opt (fun (M (name, _, _, _, _)) -> name = "main") ms with
    | Some main_method -> eval_method main_method global_env
    | None -> raise (RuntimeError "Main method not found")

File "[17]", line 7, characters 34-38:
7 |                                 | _, _ -> raise (RuntimeError "Not integer values"))
                                      ^^^^
Warning 11 [redundant-case]: this match case is unused.
File "[17]", line 10, characters 34-38:
10 |                                 | _, _ -> raise (RuntimeError "Not integer values"))
                                       ^^^^
Warning 11 [redundant-case]: this match case is unused.
File "[17]", line 13, characters 34-38:
13 |                                 | _, _ -> raise (RuntimeError "Not integer values"))
                                       ^^^^
Warning 11 [redundant-case]: this match case is unused.
File "[17]", line 19, characters 34-38:
19 |                                 | _, _ -> raise (RuntimeError "Not integer values"))
                                       ^^^^
Warning 11 [redundant-case]: this match case is unused.


error: compile_error

In [9]:
let pow = M ("pow", ["x"; "y"], ["i"; "res"],
      [Assign ("res", Id "x"); Assign ("i", Numb 1);
       While (C (Less, Id "i", Id "y"),
        [Assign ("res", App ("times", [Id "res"; Id "x"]));
         Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
       Write (Id "res")],
      Some "res") 
      
let methods = [M ("pow", ["x"; "y"], ["i"; "res"],
      [Assign ("res", Id "x"); Assign ("i", Numb 1);
       While (C (Less, Id "i", Id "y"),
        [Assign ("res", App ("times", [Id "res"; Id "x"]));
         Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
       Write (Id "res")],
      Some "res");
     M ("main", [], ["a"; "b"; "x"],
      [Assign ("a", Numb 5); Assign ("b", Numb 2);
       Assign ("x", App ("pow", [Id "b"; Id "a"]));
       Ite (C (Eq, Id "x", Numb 32), [Write (Numb 1)], [Write (Numb 0)])],
      None)]

let test_method = let 
        env = eval_method pow Env.empty in 
        assert (lookup_env "pow" env = Closure (["x"; "y"], [Assign ("res", Id "x"); Assign ("i", Numb 1);
       While (C (Less, Id "i", Id "y"),
        [Assign ("res", App ("times", [Id "res"; Id "x"]));
         Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
       Write (Id "res")], Some "res"))

let test_methods = let
        env = eval_methods methods Env.empty in 
        assert (lookup_env "pow" env = Closure (["x"; "y"], [Assign ("res", Id "x"); Assign ("i", Numb 1);
       While (C (Less, Id "i", Id "y"),
        [Assign ("res", App ("times", [Id "res"; Id "x"]));
         Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
       Write (Id "res")], Some "res"))


let prog_no_arg = P [M ("main", [], ["a"; "b"; "x"],
      [Assign ("a", Numb 5); Assign ("b", Numb 2);
       Assign ("x", App ("plus", [Id "b"; Id "a"]));
       Write (Id "x")],
      None)]


let test_no_argument = eval_program prog_no_arg
(* Should print: "OUTPUT: 7." *)

let p1 : program  =
  P
    [M ("pow", ["x"; "y"], ["i"; "res"],
      [Assign ("res", Id "x"); Assign ("i", Numb 1);
       While (C (Less, Id "i", Id "y"),
        [Assign ("res", App ("times", [Id "res"; Id "x"]));
         Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
       Write (Id "res")],
      Some "res");
     M ("main", [], ["a"; "b"; "x"],
      [Assign ("a", Numb 5); Assign ("b", Numb 2);
       Assign ("x", App ("pow", [Id "b"; Id "a"]));
       Ite (C (Eq, Id "x", Numb 32), [Write (Numb 1)], [Write (Numb 0)])],
      None)]
      
let test_program = eval_program p1      
(* Should print: 
OUTPUT:32
OUTPUT:1 *)

val pow : mmethod =
  M ("pow", ["x"; "y"], ["i"; "res"],
   [Assign ("res", Id "x"); Assign ("i", Numb 1);
    While (C (Less, Id "i", Id "y"),
     [Assign ("res", App ("times", [Id "res"; Id "x"]));
      Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
    Write (Id "res")],
   Some "res")


val methods : mmethod list =
  [M ("pow", ["x"; "y"], ["i"; "res"],
    [Assign ("res", Id "x"); Assign ("i", Numb 1);
     While (C (Less, Id "i", Id "y"),
      [Assign ("res", App ("times", [Id "res"; Id "x"]));
       Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
     Write (Id "res")],
    Some "res");
   M ("main", [], ["a"; "b"; "x"],
    [Assign ("a", Numb 5); Assign ("b", Numb 2);
     Assign ("x", App ("pow", [Id "b"; Id "a"]));
     Ite (C (Eq, Id "x", Numb 32), [Write (Numb 1)], [Write (Numb 0)])],
    None)]


val test_method : unit = ()


val test_methods : unit = ()


val prog_no_arg : program =
  P
   [M ("main", [], ["a"; "b"; "x"],
     [Assign ("a", Numb 5); Assign ("b", Numb 2);
      Assign ("x", App ("plus", [Id "b"; Id "a"])); Write (Id "x")],
     None)]


val test_no_argument : value Env.t = <abstr>


val p1 : program =
  P
   [M ("pow", ["x"; "y"], ["i"; "res"],
     [Assign ("res", Id "x"); Assign ("i", Numb 1);
      While (C (Less, Id "i", Id "y"),
       [Assign ("res", App ("times", [Id "res"; Id "x"]));
        Assign ("i", App ("plus", [Id "i"; Numb 1]))]);
      Write (Id "res")],
     Some "res");
    M ("main", [], ["a"; "b"; "x"],
     [Assign ("a", Numb 5); Assign ("b", Numb 2);
      Assign ("x", App ("pow", [Id "b"; Id "a"]));
      Ite (C (Eq, Id "x", Numb 32), [Write (Numb 1)], [Write (Numb 0)])],
     None)]


val test_program : value Env.t = <abstr>
